# Introduction

This notebook does not run in **binder**. It generates data for use by subsequent notebooks. Focus is on 
ascents for January and July 2021 at Oregon Slope Base and Axial Base, respectively **osb** and **axb**. 


In [2]:
import os, sys, time, glob, warnings
from IPython.display import clear_output             # use inside loop with clear_output(wait = True) followed by print(i)
warnings.filterwarnings('ignore')
this_dir = os.getcwd()
data_dir = this_dir + '/../data'

from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors
import numpy as np, pandas as pd, xarray as xr
from numpy import datetime64 as dt64, timedelta64 as td64

# convenience functions abbreviating 'datetime64' and so on
def doy(theDatetime): return 1 + int((theDatetime - dt64(str(theDatetime)[0:4] + '-01-01')) / td64(1, 'D'))
def dt64_from_doy(year, doy): return dt64(str(year) + '-01-01') + td64(doy-1, 'D')
def day_of_month_to_string(d): return str(d) if d > 9 else '0' + str(d)

print('\nJupyter Notebook running Python {}'.format(sys.version_info[0]))


Jupyter Notebook running Python 3


In [9]:
# Function is duplicated in notebook Ocean 01 G
# If modified here: Please re-copy it there.

def ReadProfiles(fnm):
    """
    Profiles are saved by site and year as 12-tuples. Here we read only
    the datetimes (not the indices) so there are only six values. These
    are converted to Timestamps. They correspond to ascend start/end, 
    descend start/end and rest start/end.
    """
    df = pd.read_csv(fnm, usecols=["1", "3", "5", "7", "9", "11"])
    df.columns=['ascent_start', 'ascent_end', 'descent_start', 'descent_end', 'rest_start', 'rest_end']
    df['ascent_start'] = pd.to_datetime(df['ascent_start'])
    df['ascent_end'] = pd.to_datetime(df['ascent_end'])
    df['descent_start'] = pd.to_datetime(df['descent_start'])
    df['descent_end'] = pd.to_datetime(df['descent_end'])
    df['rest_start'] = pd.to_datetime(df['rest_start'])
    df['rest_end'] = pd.to_datetime(df['rest_end'])
    return df

df_osb2021 = ReadProfiles(os.getcwd()+"/profiles/osb2021.csv")
df_axb2021 = ReadProfiles(os.getcwd()+"/profiles/axb2021.csv")



In [10]:
df_osb2021

,ascent_start,ascent_end,descent_start,descent_end,rest_start,rest_end
0,2021-01-01 00:27:00,2021-01-01 01:30:00,2021-01-01 01:30:00,2021-01-01 02:02:00,2021-01-01 02:02:00,2021-01-01 02:42:00
1,2021-01-01 02:42:00,2021-01-01 03:46:00,2021-01-01 03:46:00,2021-01-01 04:18:00,2021-01-01 04:18:00,2021-01-01 04:52:00
2,2021-01-01 04:52:00,2021-01-01 05:56:00,2021-01-01 05:56:00,2021-01-01 06:29:00,2021-01-01 06:29:00,2021-01-01 07:22:00
3,2021-01-01 07:22:00,2021-01-01 08:27:00,2021-01-01 08:27:00,2021-01-01 09:56:00,2021-01-01 09:56:00,2021-01-01 11:12:00
4,2021-01-01 11:12:00,2021-01-01 12:16:00,2021-01-01 12:16:00,2021-01-01 12:49:00,2021-01-01 12:49:00,2021-01-01 13:32:00
...,...,...,...,...,...,...
1850,2021-08-02 07:22:00,2021-08-02 08:29:00,2021-08-02 08:29:00,2021-08-02 10:04:00,2021-08-02 10:04:00,2021-08-02 11:12:00
1851,2021-08-02 11:12:00,2021-08-02 12:18:00,2021-08-02 12:18:00,2021-08-02 12:52:00,2021-08-02 12:52:00,2021-08-02 13:32:00
1852,2021-08-02 13:32:00,2021-08-02 14:38:00,2021-08-02 14:38:00,2021-08-02 15:12:00,2021-08-02 15:12:00,2021-08-02 15:47:00
1853,2021-08-02 15:47:00,2021-08-02 16:53:00,2021-08-02 16:53:00,2021-08-02 17:27:00,2021-08-02 17:27:00,2021-08-02 18:12:00


In [70]:
# generate four dataframes of profiles: (osb, axb) x (Jan 2021, Jul 2021)

tRangeJan1, tRangeFeb1 = dt64('2021-01-01'), dt64('2021-02-01')
tRangeJul1, tRangeAug1 = dt64('2021-07-01'), dt64('2021-08-01')
df_osb_jan = df_osb2021[(df_osb2021['ascent_start'] > tRangeJan1) & (df_osb2021['ascent_start'] < tRangeFeb1)]
df_osb_jul = df_osb2021[(df_osb2021['ascent_start'] > tRangeJul1) & (df_osb2021['ascent_start'] < tRangeAug1)]
df_axb_jan = df_axb2021[(df_axb2021['ascent_start'] > tRangeJan1) & (df_axb2021['ascent_start'] < tRangeFeb1)]
df_axb_jul = df_axb2021[(df_axb2021['ascent_start'] > tRangeJul1) & (df_axb2021['ascent_start'] < tRangeAug1)]

# print("\n\nosb january:\n\n", df_osb_jan)
# print("\n\nosb july:\n\n", df_osb_jul)
# print("\n\naxb january:\n\n", df_axb_jan)
# print("\n\naxb july:\n\n", df_axb_jul)

# this works: 
# print((dt64(df_osb_jan['ascent_end'][0]) - dt64(df_osb_jan['ascent_start'][0])).astype('timedelta64[m]'))

# this works to print an example time-of-ascent
# gap_in_time = dt64(df_osb_jan['ascent_end'][0]) - dt64(df_osb_jan['ascent_start'][0])
# print(gap_in_time.astype('timedelta64[m]'))
# print(type(gap_in_time))


In [74]:
# Look for anomalous (too long / too brief) ascent durations in one of our four cases

# Calculate mean ascent duration (61 minutes) for osb/january
ascdurs_osb_jan_series = df_osb_jan['ascent_end'] - df_osb_jan['ascent_start']
n_ascents = len(ascdurs_osb_jan_series)
ascdurs_osb_jan_mean = td64(ascdurs_osb_jan_series.mean())

# Prepare to scan all 256 profiles for anomalous time durations: outside (mean +- threshold) minutes
mdenom = td64(1, "m")
mean_ascent_time = ascdurs_osb_jan_mean/mdenom            # converts to float (in minutes)
the_threshold = 5.

# diagnostic printouts
print(ascdurs_osb_jan_mean.astype('timedelta64[m]'), '\n\n')
for i in range(n_ascents):
    if ascdurs_osb_jan_series[i]/mdenom > mean_ascent_time + the_threshold:
        print('Hi: ', i, ascdurs_osb_jan_series[i]/mdenom)
    if ascdurs_osb_jan_series[i]/mdenom < mean_ascent_time - the_threshold:
        print('                      Lo: ', i, ascdurs_osb_jan_series[i]/mdenom)
        

61 minutes 


                      Lo:  11 56.0
                      Lo:  12 54.0
                      Lo:  13 56.0
                      Lo:  14 52.0
                      Lo:  15 55.0
                      Lo:  16 56.0
                      Lo:  27 52.0
                      Lo:  28 55.0
                      Lo:  29 54.0
                      Lo:  30 54.0
                      Lo:  31 53.0
                      Lo:  32 55.0
                      Lo:  39 53.0
                      Lo:  40 56.0
                      Lo:  41 56.0
                      Lo:  44 55.0
                      Lo:  45 52.0
                      Lo:  46 56.0
                      Lo:  57 56.0
                      Lo:  71 55.0
                      Lo:  85 47.0
                      Lo:  86 41.0
                      Lo:  87 48.0
                      Lo:  88 47.0
                      Lo:  89 52.0
                      Lo:  90 55.0
                      Lo:  104 54.0
                      Lo:  106 55.0
    

In [76]:
# change the four loop to index i
# pre-build a list of these four dataframes
# pre-build a list of source filenames
for source in [df_osb_jan, df_osb_jul, df_axb_jan, df_axb_jul]:
    # print(source.shape) -> (256, 6), (271, 6), (254, 6), (278, 6)
    print("oy")
    read the source data into an xarray Dataset
    loop over ascents
        loop over the minutes
            get the pressure/depth values as an xarray dimension
    backscatter
    cdom
    chlora
    doxygen
    ?nitrate?
    par
    pco2
    ?ph?
    salinity
    spkir
    temperature
    

oy
oy
oy
oy
